Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


### Accuracy function

In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with **two convolutional layers, followed by one fully connected layer**. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

- tf.nn.conv2d() 详细用法: [conv2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)

<span style = "color:blue">some tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu = None, data_format = None, name = None)</span>

For the most common case of the same horizontal and vertices strides, <span style = "color:blue"> strides = [1, stride, stride, 1]</span>.

- [tf.nn.relu](https://www.tensorflow.org/api_docs/python/tf/nn/relu)

<span style = "color:blue"> tf.nn.relu(features, name = None)</span>

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # 1 training data
    tf_train_dataset = tf.placeholder(
    tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    # training labels
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset) # valid data
    tf_test_dataset = tf.constant(test_dataset) # test data
  

    # 2 Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  

    # 3 Model
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    
    # 4 Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    
    # 5 Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  

    # 6 Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.720697
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.367468
Minibatch accuracy: 62.5%
Validation accuracy: 55.5%
Minibatch loss at step 100: 1.013464
Minibatch accuracy: 68.8%
Validation accuracy: 73.2%
Minibatch loss at step 150: 0.460501
Minibatch accuracy: 81.2%
Validation accuracy: 74.1%
Minibatch loss at step 200: 0.469433
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.448816
Minibatch accuracy: 68.8%
Validation accuracy: 76.6%
Minibatch loss at step 300: 0.680893
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 350: 0.476321
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.702988
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 450: 0.671029
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.559463
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

- [tf.nn.max_pool](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool)

<span style = "color:blue"> tf.nn.max_pool(value, ksize, strides, padding, data_format = 'NHWC', name = None)</span>

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # 1 training data
    tf_train_dataset = tf.placeholder(
    tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    # training labels
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset) # valid data
    tf_test_dataset = tf.constant(test_dataset) # test data
  

    # 2 Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  

    # 3 Model
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding = 'SAME')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')
        
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding = 'SAME')
        bias2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')
        
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    
    # 4 Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    
    # 5 Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  

    # 6 Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.510817
Minibatch accuracy: 25.0%
Validation accuracy: 10.1%
Minibatch loss at step 50: 2.147055
Minibatch accuracy: 6.2%
Validation accuracy: 11.9%
Minibatch loss at step 100: 1.519373
Minibatch accuracy: 56.2%
Validation accuracy: 52.9%
Minibatch loss at step 150: 0.605193
Minibatch accuracy: 75.0%
Validation accuracy: 60.5%
Minibatch loss at step 200: 0.948872
Minibatch accuracy: 62.5%
Validation accuracy: 73.7%
Minibatch loss at step 250: 1.560502
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.708271
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.481250
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.811856
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.560896
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.568321
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---
1. The CNN below is loosely inspired by the LeNet5 architecture.

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # 1 Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  

    # 2 Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
        [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # 3 Model.
    def model(data):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # C3 input 12 x 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F6 input 4 x 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # 4 Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    
    # 5 Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  

    # 6 Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 5001 # 为了简便，不训练太久

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.665139
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.607986
Minibatch accuracy: 37.5%
Validation accuracy: 40.0%
Minibatch loss at step 100: 1.127173
Minibatch accuracy: 62.5%
Validation accuracy: 64.3%
Minibatch loss at step 150: 0.656384
Minibatch accuracy: 62.5%
Validation accuracy: 64.0%
Minibatch loss at step 200: 1.048097
Minibatch accuracy: 56.2%
Validation accuracy: 70.4%
Minibatch loss at step 250: 1.381778
Minibatch accuracy: 68.8%
Validation accuracy: 74.8%
Minibatch loss at step 300: 0.848730
Minibatch accuracy: 68.8%
Validation accuracy: 74.5%
Minibatch loss at step 350: 0.515433
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 400: 1.090250
Minibatch accuracy: 56.2%
Validation accuracy: 76.0%
Minibatch loss at step 450: 0.777323
Minibatch accuracy: 68.8%
Validation accuracy: 78.3%
Minibatch loss at step 500: 0.582095
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Mi

The test accuracy is 92.4%.

2.Use dropout and learning rate decay

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

    # 1 Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
  
    # 2 Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
        [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  

    # 3 Model.
    def model(data, keep_prob):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # C3 input 12 x 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F5 input 4 x 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # F6
        drop5 = tf.nn.dropout(hidden5, keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
        drop6 = tf.nn.dropout(hidden6, keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_biases
  
    # 4 Training computation.
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    
    # 5 Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  

    # 6 Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [16]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3.059666
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.055718
Minibatch accuracy: 25.0%
Validation accuracy: 33.9%
Minibatch loss at step 100: 1.932189
Minibatch accuracy: 18.8%
Validation accuracy: 38.2%
Minibatch loss at step 150: 1.650250
Minibatch accuracy: 25.0%
Validation accuracy: 38.2%
Minibatch loss at step 200: 1.219723
Minibatch accuracy: 50.0%
Validation accuracy: 55.4%
Minibatch loss at step 250: 1.898571
Minibatch accuracy: 37.5%
Validation accuracy: 59.1%
Minibatch loss at step 300: 1.065329
Minibatch accuracy: 50.0%
Validation accuracy: 59.9%
Minibatch loss at step 350: 1.422922
Minibatch accuracy: 43.8%
Validation accuracy: 69.6%
Minibatch loss at step 400: 1.362604
Minibatch accuracy: 43.8%
Validation accuracy: 69.2%
Minibatch loss at step 450: 1.589878
Minibatch accuracy: 43.8%
Validation accuracy: 71.6%
Minibatch loss at

The accuracy is 91%. This net has many meta parameters.